In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
cd drive

In [0]:
cd My Drive

In [0]:
cd Colab Notebooks

In [0]:
import numpy
import pandas as pd
import tensorflow as tf
import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import MaxPooling2D,Dropout,Lambda,Concatenate
from keras.models import Model
K.clear_session()

In [0]:
#Hyperparameters
minibatch_size=256
ch=[32,64,64]
f=3
s=2
maxp_f=[2,2,2]
maxp_s=2
pad=['same','same','same']
den=[1024,1024,128,6]
epochs=50
labels=["3","4","5","6","7","8"]

In [0]:
train_data=ImageDataGenerator(rescale=1./255).flow_from_directory(
        directory='./DATA/train',
        target_size=(256,256),
        color_mode='rgb',
        classes=labels,
        batch_size=minibatch_size,
        class_mode='categorical',
        shuffle=True)
(X_train,Y_train)=train_data[0]

Loading Training data in numpy array format.

In [0]:
#Convolutional layer with Conv2D,BatchNormalization,Activation of Relu and MaxPooling
def conv_layer_with_pool(X_prev,l):
    X=Conv2D(ch[l],kernel_size=(f,f),strides=(s,s),padding=pad[l],name="conv"+str(l))(X_prev)
    X=BatchNormalization(axis=3,name="BN"+str(l))(X)
    X=Activation('relu')(X)
    X_inc=X
    if maxp_f[l] != 0:
        X=MaxPooling2D((maxp_f[l],maxp_f[l]),strides=maxp_s)(X)
    #X=Dropout(convdo[l])(X)
    return X

In [0]:

def neural_net(input_shape=(256,256,3),output_size=6):
    X_input=Input(input_shape)
    X=X_input
    for l in range(len(ch)):
        X_prev=X
        X=conv_layer_with_pool(X_prev,l)
    X=Flatten()(X)
    for l in range(len(den)-1):
        X=Dense(den[l],activation="relu",name="fc"+str(l))(X)
        #X=Dropout(do[l])(X)
    X=Dense(den[len(den)-1],activation="softmax",name="fc"+str(len(den)-1))(X)
    model=Model(inputs=X_input,outputs=X,name="EgdeDetection")
    return model

In [0]:
model=neural_net()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(
        train_data,
        steps_per_epoch=32,
        epochs=epochs)
preds=model.evaluate(X_train,Y_train)
print("Training Loss: "+str(preds[0]))
print("Training Accuracy: "+str(preds[1]))